In [1]:
import os

In [2]:
pwd

'/scratch/Shaz/subsidiary_work/Text_Summarizer-Project/research'

In [3]:
os.chdir('../')

In [4]:
pwd

'/scratch/Shaz/subsidiary_work/Text_Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.__init__ import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self):

        self.config = read_yaml(Path("config/config.yaml"))
        self.params = read_yaml(Path("params.yaml"))

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.evaluation_strategy,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        ) 

        return model_trainer_config
    

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2024-02-26 15:33:50,059: INFO: config:PyTorch version 2.2.1 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # We can use either of the two training arguments 

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps, 
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=20, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01,logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )
        
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                          tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["train"],
                        eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()

        # Save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))

        # Save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-26 15:33:50,526: INFO: __init__:yaml file: config/config.yaml loaded succesfully]
[2024-02-26 15:33:50,528: INFO: __init__:yaml file: params.yaml loaded succesfully]
[2024-02-26 15:33:50,529: INFO: __init__:created directory at: artifacts]
[2024-02-26 15:33:50,530: INFO: __init__:created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2024-02-26 15:34:19,723: WARNING: logging:Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.]


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [19]:
# Follow the below procedure if you face any error:

# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [20]:
!pip install transformers accelerate

In [3]:
import torch
print("cuda" if torch.cuda.is_available() else "cpu")

cuda


In [4]:
import sys
print(sys.prefix)

/scratch/Shaz/miniconda3/envs/textS


In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA is available")
    print("PyTorch version:", torch.__version__)
    print("CUDA version:", torch.version.cuda)
    print("cuDNN version:", torch.backends.cudnn.version())
    print("CUDA device count:", torch.cuda.device_count())
    print("CUDA devices:")
    for i in range(torch.cuda.device_count()):
        print(f"  - {torch.cuda.get_device_name(i)} (Device {i})")
else:
    print("CUDA is not available")

CUDA is available
PyTorch version: 2.2.1+cu121
CUDA version: 12.1
cuDNN version: 8902
CUDA device count: 1
CUDA devices:
  - NVIDIA GeForce RTX 3090 (Device 0)


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3090


In [21]:
pwd

'/home/shazam/Text_Sumamrizer_Project'